# README.

This is just a sandbox. No production code.

In [24]:
import dbutils
import glob2 as glob
import os

glob_search_path = os.path.join("/whhdata/person/**/*.jpg")
print("Searching at {}... This might take a while!".format(glob_search_path))
jpg_paths = glob.glob(glob_search_path) # TODO make this work again!
print("Found {} JPGs.".format(len(jpg_paths)))

Searching at /whhdata/person/**/*.jpg... This might take a while!
Found 16329 JPGs.


In [ ]:
import random
import dbutils

main_connector = dbutils.connect_to_main_database()

In [26]:
for path in jpg_paths:
#for _ in range(10):
    #path = random.choice(jpg_paths)
    path_split = path.split("/")

    qrcode = path_split[3]
    timestamp = path_split[-1].split("_")[-3]
    #print(qrcode)
    #print(timestamp)
    #print("")
    
    # Todo get id of measurement.
    threshold = int(60 * 60 * 24 * 1000)
    sql_statement = dbutils.create_select_statement("measurements", ["qrcode"], [qrcode])
    sql_statement = ""
    sql_statement += "SELECT id, timestamp"
    sql_statement += " FROM measurements WHERE"
    sql_statement += " qrcode = '{}'".format(qrcode)
    sql_statement += " AND type = 'manual'"
    sql_statement += " AND ABS(timestamp - {}) < {}".format(timestamp, threshold)
    sql_statement += ";"
    #print(sql_statement)
    results = main_connector.execute(sql_statement, fetch_all=True)
    if len(results) != 0:
        print(path)
        print(timestamp, "\tjpg path")
        print(results[0][1], "\tmeasurement")
        print(abs(int(timestamp) - int(results[0][1])), "\tdifference")
        print("")

/whhdata/person/MH_WHH_0153/measurements/1537860868501/rgb/rgb_MH_WHH_0153_1537860868501_104_95405.92970875901.jpg
1537860868501 	jpg path
1537861056986 	measurement
188485 	difference

/whhdata/person/MH_WHH_0153/measurements/1537860868501/rgb/rgb_MH_WHH_0153_1537860868501_104_95405.99664075901.jpg
1537860868501 	jpg path
1537861056986 	measurement
188485 	difference

/whhdata/person/MH_WHH_0153/measurements/1537860868501/rgb/rgb_MH_WHH_0153_1537860868501_104_95406.130336759.jpg
1537860868501 	jpg path
1537861056986 	measurement
188485 	difference

/whhdata/person/MH_WHH_0153/measurements/1537860868501/rgb/rgb_MH_WHH_0153_1537860868501_104_95406.230664759.jpg
1537860868501 	jpg path
1537861056986 	measurement
188485 	difference

/whhdata/person/MH_WHH_0153/measurements/1537860868501/rgb/rgb_MH_WHH_0153_1537860868501_104_95406.29754075901.jpg
1537860868501 	jpg path
1537861056986 	measurement
188485 	difference

/whhdata/person/MH_WHH_0153/measurements/1537860868501/rgb/rgb_MH_WHH_0153

/whhdata/person/MH_WHH_0153/measurements/1537860868501/rgb/rgb_MH_WHH_0153_1537860868501_107_95419.139044052.jpg
1537860868501 	jpg path
1537861056986 	measurement
188485 	difference

/whhdata/person/MH_WHH_0153/measurements/1537860868501/rgb/rgb_MH_WHH_0153_1537860868501_107_95419.272347052.jpg
1537860868501 	jpg path
1537861056986 	measurement
188485 	difference

/whhdata/person/MH_WHH_0153/measurements/1537860868501/rgb/rgb_MH_WHH_0153_1537860868501_107_95419.405659052.jpg
1537860868501 	jpg path
1537861056986 	measurement
188485 	difference

/whhdata/person/MH_WHH_0153/measurements/1537860868501/rgb/rgb_MH_WHH_0153_1537860868501_107_95419.538959052.jpg
1537860868501 	jpg path
1537861056986 	measurement
188485 	difference

/whhdata/person/MH_WHH_0153/measurements/1537860868501/rgb/rgb_MH_WHH_0153_1537860868501_107_95419.672272052.jpg
1537860868501 	jpg path
1537861056986 	measurement
188485 	difference

/whhdata/person/MH_WHH_0153/measurements/1537860868501/rgb/rgb_MH_WHH_0153_15378

/whhdata/person/MH_WHH_0004/measurements/1536913535697/rgb/rgb_MH_WHH_0004_1536913535697_107_7361.380521759001.jpg
1536913535697 	jpg path
1536913613101 	measurement
77404 	difference

/whhdata/person/MH_WHH_0004/measurements/1536913535697/rgb/rgb_MH_WHH_0004_1536913535697_107_7361.4808587590005.jpg
1536913535697 	jpg path
1536913613101 	measurement
77404 	difference

/whhdata/person/MH_WHH_0004/measurements/1536913535697/rgb/rgb_MH_WHH_0004_1536913535697_107_7361.6145927590005.jpg
1536913535697 	jpg path
1536913613101 	measurement
77404 	difference

/whhdata/person/MH_WHH_0004/measurements/1536913535697/rgb/rgb_MH_WHH_0004_1536913535697_107_7361.748372759001.jpg
1536913535697 	jpg path
1536913613101 	measurement
77404 	difference

/whhdata/person/MH_WHH_0004/measurements/1536913535697/rgb/rgb_MH_WHH_0004_1536913535697_107_7361.882149759001.jpg
1536913535697 	jpg path
1536913613101 	measurement
77404 	difference

/whhdata/person/MH_WHH_0004/measurements/1536913535697/rgb/rgb_MH_WHH_000

/whhdata/person/MH_WHH_0004/measurements/1536913535697/rgb/rgb_MH_WHH_0004_1536913535697_110_7377.966723759.jpg
1536913535697 	jpg path
1536913613101 	measurement
77404 	difference

/whhdata/person/MH_WHH_0004/measurements/1536913535697/rgb/rgb_MH_WHH_0004_1536913535697_110_7378.100472759001.jpg
1536913535697 	jpg path
1536913613101 	measurement
77404 	difference

/whhdata/person/MH_WHH_0004/measurements/1536913535697/rgb/rgb_MH_WHH_0004_1536913535697_110_7378.267670759001.jpg
1536913535697 	jpg path
1536913613101 	measurement
77404 	difference

/whhdata/person/MH_WHH_0004/measurements/1536913535697/rgb/rgb_MH_WHH_0004_1536913535697_110_7378.401412759001.jpg
1536913535697 	jpg path
1536913613101 	measurement
77404 	difference

/whhdata/person/MH_WHH_0004/measurements/1536913535697/rgb/rgb_MH_WHH_0004_1536913535697_110_7378.568612759001.jpg
1536913535697 	jpg path
1536913613101 	measurement
77404 	difference

/whhdata/person/MH_WHH_0004/measurements/1536913535697/rgb/rgb_MH_WHH_0004_153

/whhdata/person/MH_WHH_0001/measurements/1536913319075/rgb/rgb_MH_WHH_0001_1536913319075_107_7144.343445052001.jpg
1536913319075 	jpg path
1536913412003 	measurement
92928 	difference

/whhdata/person/MH_WHH_0001/measurements/1536913319075/rgb/rgb_MH_WHH_0001_1536913319075_107_7144.443434052.jpg
1536913319075 	jpg path
1536913412003 	measurement
92928 	difference

/whhdata/person/MH_WHH_0001/measurements/1536913319075/rgb/rgb_MH_WHH_0001_1536913319075_107_7144.543402052001.jpg
1536913319075 	jpg path
1536913412003 	measurement
92928 	difference

/whhdata/person/MH_WHH_0001/measurements/1536913319075/rgb/rgb_MH_WHH_0001_1536913319075_107_7144.643380052001.jpg
1536913319075 	jpg path
1536913412003 	measurement
92928 	difference

/whhdata/person/MH_WHH_0001/measurements/1536913319075/rgb/rgb_MH_WHH_0001_1536913319075_107_7144.776703052001.jpg
1536913319075 	jpg path
1536913412003 	measurement
92928 	difference

/whhdata/person/MH_WHH_0001/measurements/1536913319075/rgb/rgb_MH_WHH_0001_153

...

/whhdata/person/MH_WHH_1022/measurements/1543827517529/rgb/rgb_MH_WHH_1022_1543827517529_107_8403.384369052.jpg
1543827517529 	jpg path
1543827720391 	measurement
202862 	difference

/whhdata/person/MH_WHH_1022/measurements/1543827517529/rgb/rgb_MH_WHH_1022_1543827517529_107_8403.517669052.jpg
1543827517529 	jpg path
1543827720391 	measurement
202862 	difference

/whhdata/person/MH_WHH_1022/measurements/1543827517529/rgb/rgb_MH_WHH_1022_1543827517529_107_8403.584326052001.jpg
1543827517529 	jpg path
1543827720391 	measurement
202862 	difference

/whhdata/person/MH_WHH_1022/measurements/1543827517529/rgb/rgb_MH_WHH_1022_1543827517529_107_8403.684300052.jpg
1543827517529 	jpg path
1543827720391 	measurement
202862 	difference

/whhdata/person/MH_WHH_1022/measurements/1543827517529/rgb/rgb_MH_WHH_1022_1543827517529_107_8403.784280052001.jpg
1543827517529 	jpg path
1543827720391 	measurement
202862 	difference

/whhdata/person/MH_WHH_1022/measurements/1543827517529/rgb/rgb_MH_WHH_1022_1543

/whhdata/person/MH_WHH_1022/measurements/1543827517529/rgb/rgb_MH_WHH_1022_1543827517529_110_8434.044223052.jpg
1543827517529 	jpg path
1543827720391 	measurement
202862 	difference

/whhdata/person/MH_WHH_1022/measurements/1543827517529/rgb/rgb_MH_WHH_1022_1543827517529_110_8434.144132052.jpg
1543827517529 	jpg path
1543827720391 	measurement
202862 	difference

/whhdata/person/MH_WHH_1022/measurements/1543827517529/rgb/rgb_MH_WHH_1022_1543827517529_110_8434.244113052.jpg
1543827517529 	jpg path
1543827720391 	measurement
202862 	difference

/whhdata/person/MH_WHH_1022/measurements/1543827517529/rgb/rgb_MH_WHH_1022_1543827517529_110_8434.344096052.jpg
1543827517529 	jpg path
1543827720391 	measurement
202862 	difference

/whhdata/person/MH_WHH_1022/measurements/1543827517529/rgb/rgb_MH_WHH_1022_1543827517529_110_8434.410741052001.jpg
1543827517529 	jpg path
1543827720391 	measurement
202862 	difference

/whhdata/person/MH_WHH_1022/measurements/1543827517529/rgb/rgb_MH_WHH_1022_1543827

/whhdata/person/MH_WHH_1026/measurements/1543312995458/rgb/rgb_MH_WHH_1026_1543312995458_107_8092.539591759.jpg
1543312995458 	jpg path
1543313100887 	measurement
105429 	difference

/whhdata/person/MH_WHH_1026/measurements/1543312995458/rgb/rgb_MH_WHH_1026_1543312995458_107_8092.639909759.jpg
1543312995458 	jpg path
1543313100887 	measurement
105429 	difference

/whhdata/person/MH_WHH_1026/measurements/1543312995458/rgb/rgb_MH_WHH_1026_1543312995458_107_8092.773667759.jpg
1543312995458 	jpg path
1543313100887 	measurement
105429 	difference

/whhdata/person/MH_WHH_1026/measurements/1543312995458/rgb/rgb_MH_WHH_1026_1543312995458_107_8092.907431759.jpg
1543312995458 	jpg path
1543313100887 	measurement
105429 	difference

/whhdata/person/MH_WHH_1026/measurements/1543312995458/rgb/rgb_MH_WHH_1026_1543312995458_107_8093.007742759.jpg
1543312995458 	jpg path
1543313100887 	measurement
105429 	difference

/whhdata/person/MH_WHH_1026/measurements/1543312995458/rgb/rgb_MH_WHH_1026_1543312995

/whhdata/person/MH_WHH_1032/measurements/1543299884019/rgb/rgb_MH_WHH_1032_1543299884019_104_264.865879759.jpg
1543299884019 	jpg path
1543299961614 	measurement
77595 	difference

/whhdata/person/MH_WHH_1032/measurements/1543299884019/rgb/rgb_MH_WHH_1032_1543299884019_104_264.966045759.jpg
1543299884019 	jpg path
1543299961614 	measurement
77595 	difference

/whhdata/person/MH_WHH_1032/measurements/1543299884019/rgb/rgb_MH_WHH_1032_1543299884019_104_265.099791759.jpg
1543299884019 	jpg path
1543299961614 	measurement
77595 	difference

/whhdata/person/MH_WHH_1032/measurements/1543299884019/rgb/rgb_MH_WHH_1032_1543299884019_104_265.166687759.jpg
1543299884019 	jpg path
1543299961614 	measurement
77595 	difference

/whhdata/person/MH_WHH_1032/measurements/1543299884019/rgb/rgb_MH_WHH_1032_1543299884019_104_265.300426759.jpg
1543299884019 	jpg path
1543299961614 	measurement
77595 	difference

/whhdata/person/MH_WHH_1032/measurements/1543299884019/rgb/rgb_MH_WHH_1032_1543299884019_104_26

/whhdata/person/MH_WHH_1032/measurements/1543299884019/rgb/rgb_MH_WHH_1032_1543299884019_110_283.65893475900003.jpg
1543299884019 	jpg path
1543299961614 	measurement
77595 	difference

/whhdata/person/MH_WHH_1032/measurements/1543299884019/rgb/rgb_MH_WHH_1032_1543299884019_110_283.759252759.jpg
1543299884019 	jpg path
1543299961614 	measurement
77595 	difference

/whhdata/person/MH_WHH_1032/measurements/1543299884019/rgb/rgb_MH_WHH_1032_1543299884019_110_283.89319575900004.jpg
1543299884019 	jpg path
1543299961614 	measurement
77595 	difference

/whhdata/person/MH_WHH_1032/measurements/1543299884019/rgb/rgb_MH_WHH_1032_1543299884019_110_283.993332759.jpg
1543299884019 	jpg path
1543299961614 	measurement
77595 	difference

/whhdata/person/MH_WHH_1032/measurements/1543299884019/rgb/rgb_MH_WHH_1032_1543299884019_110_284.12714375900003.jpg
1543299884019 	jpg path
1543299961614 	measurement
77595 	difference

/whhdata/person/MH_WHH_1032/measurements/1543299884019/rgb/rgb_MH_WHH_1032_15432

In [19]:
int(60 * 60 * 24 * 1000)

86400000

In [ ]:
14617848875
   86400000

In [31]:
main_connector = dbutils.connect_to_main_database()
sql_statement = "ALTER TABLE measurements RENAME COLUMN \"weight_cms' TO 'weight_kgs'"
result = main_connector.execute(sql_statement)
print(result)


ProgrammingError: syntax error at or near "'weight_cms'"
LINE 1: ALTER TABLE measurements RENAME COLUMN 'weight_cms' TO 'weig...
                                               ^
